In [20]:
import sqlite3
import pandas as pd

## Load Appointment dataset to Dataframe

In [21]:
appointment_df = pd.read_csv("appointment_data.csv")
appointment_df.head(5)

,Appointment,PatientID,DoctorID,Department,Date,Time,Status
0,A9986,P50298,D8,Cardiology,"Thursday, May 9, 2024",8:00,Completed
1,A9964,P95475,D1,Neurology,"Sunday, January 7, 2024",16:00,No Show
2,A9961,P76847,D24,Orthopedics,"Sunday, February 25, 2024",9:30,Cancelled
3,A9946,P82099,D16,Orthopedics,"Tuesday, October 29, 2024",15:30,No Show
4,A9814,P47436,D17,Cardiology,"Saturday, March 16, 2024",14:30,Rescheduled


## Load Resource dataset to Dataframe

In [22]:
resource_df = pd.read_csv("resource_data.csv")
resource_df.head(5)

,ResourceID,Department,ResourceType,UsageHours,Date,Availability
0,R5487,Cardiology,Nurse,2,"Sunday, July 28, 2024",Unavailable
1,R8097,Orthopedics,Doctor,9,"Tuesday, September 10, 2024",Unavailable
2,R2557,General Medicine,Room,5,"Tuesday, December 17, 2024",Under Maintenance
3,R9024,Neurology,Doctor,9,"Tuesday, July 9, 2024",Unavailable
4,R2629,Oncology,Doctor,4,"Wednesday, March 6, 2024",Unavailable


## Load Feedback dataset to Dataframe

In [23]:
feedback_df = pd.read_csv("feedback_data.csv")
feedback_df.head(5)

,FeedbackID,PatientID,Department,Feedback_Score,Comments,WaitTime
0,F1000,P96099,Cardiology,4,Delayed response,45
1,F1001,P16262,Orthopedics,3,Needs improvement,45
2,F1003,P55235,Neurology,9,Amazing team,30
3,F1004,P93372,Pediatric,8,Great doctors,45
4,F1005,P84444,Pediatric,5,Satisfactory,45


## Connect to Database and Create Appointment table

In [24]:
conn = sqlite3.connect("my_database.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS appointments (
    Appointment TEXT,
    PatientID INTEGER,
    DoctorID INTEGER,
    Department TEXT,
    Date TEXT,
    Time TEXT,
    Status TEXT
)
""")
conn.commit()
appointment_df.to_sql("appointments", conn, if_exists="append", index=False)
pd.read_sql_query("SELECT * FROM appointments LIMIT 10", conn)

,Appointment,PatientID,DoctorID,Department,Date,Time,Status
0,A7515,P16262,D1,Orthopedics,9/10/2024,13:00,No Show
1,A7537,P99601,D30,General Medicine,12/17/2024,15:00,Completed
2,A8866,P55235,D30,Neurology,7/9/2024,15:30,Rescheduled
3,A3039,P93372,D23,Pediatric,8/12/2024,18:00,Cancelled
4,A3237,P84444,D1,Pediatric,3/6/2024,17:30,Rescheduled
5,A8127,P41013,D24,Neurology,4/1/2024,11:00,No Show
6,A7025,P17235,D10,Neurology,10/16/2024,15:00,No Show
7,A8462,P35768,D8,Cardiology,9/23/2024,15:30,Completed
8,A9018,P28832,D14,Orthopedics,4/13/2024,13:30,Rescheduled
9,A7791,P29680,D28,Orthopedics,6/17/2024,8:30,No Show


## Create Feedback table

In [25]:
cursor.execute("Drop table IF Exists feedback")

#Recreate with the correct schema
cursor.execute("""
CREATE TABLE feedback (
    FeedbackID INTEGER,
    PatientID INTEGER,
    Department TEXT,
    Feedback_Score INTEGER,
    Comments TEXT,
    WaitTime TEXT
)
""")
conn.commit()

# Insert data
feedback_df.to_sql("feedback", conn, if_exists="append", index=False)

# Verify
pd.read_sql_query("SELECT * FROM feedback LIMIT 5", conn)

,FeedbackID,PatientID,Department,Feedback_Score,Comments,WaitTime
0,F1000,P96099,Cardiology,4,Delayed response,45
1,F1001,P16262,Orthopedics,3,Needs improvement,45
2,F1003,P55235,Neurology,9,Amazing team,30
3,F1004,P93372,Pediatric,8,Great doctors,45
4,F1005,P84444,Pediatric,5,Satisfactory,45


## Create Resource table

In [26]:
cursor.execute("drop table if exists resource")
cursor.execute("""
create table resource (
ResourceID text,
Department text,
ResourceType text,
UsageHours integer,
Date text,
Availability text
)
""")
conn.commit()

resource_df.to_sql("resource", conn, if_exists= "append", index=False)

pd.read_sql_query("Select * from resource limit 5", conn)

,ResourceID,Department,ResourceType,UsageHours,Date,Availability
0,R5487,Cardiology,Nurse,2,"Sunday, July 28, 2024",Unavailable
1,R8097,Orthopedics,Doctor,9,"Tuesday, September 10, 2024",Unavailable
2,R2557,General Medicine,Room,5,"Tuesday, December 17, 2024",Under Maintenance
3,R9024,Neurology,Doctor,9,"Tuesday, July 9, 2024",Unavailable
4,R2629,Oncology,Doctor,4,"Wednesday, March 6, 2024",Unavailable


In [27]:
pd.read_sql_query("""
select appointment, patientid 
from appointments 
where appointment = 'A7515'""", conn)

,Appointment,PatientID
0,A7515,P16262
1,A7515,P16262
2,A7515,P16262
3,A7515,P16262
4,A7515,P16262
5,A7515,P16262
6,A7515,P16262
7,A7515,P16262
8,A7515,P16262
9,A7515,P16262


## What is the appointment completion 
# vs. cancellation/no-show rate by department?

In [28]:
pd.read_sql_query("""SELECT 
  Department,
  Status,
  COUNT(*) AS AppointmentCount,
  COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Department) AS Percentage
FROM Appointments
WHERE Department != 'None'
GROUP BY Department, Status""", conn)

,Department,Status,AppointmentCount,Percentage
0,Cardiology,Cancelled,121,30.024814
1,Cardiology,Completed,141,34.987593
2,Cardiology,No Show,55,13.647643
3,Cardiology,Rescheduled,86,21.339950
4,General Medicine,None,27,8.681672
5,General Medicine,Cancelled,66,21.221865
6,General Medicine,Completed,99,31.832797
7,General Medicine,No Show,64,20.578778
8,General Medicine,Rescheduled,55,17.684887
9,Neurology,None,9,2.205882


## How many appointments does 
# each doctor handle per day?

In [29]:
pd.read_sql_query("""SELECT 
    DoctorID,
    AVG(DailyCount) AS AvgDailyAppointments
FROM (
    SELECT 
        DoctorID,
        Date,
        COUNT(*) AS DailyCount
    FROM Appointments
    GROUP BY DoctorID, Date
) AS DailyStats
WHERE DoctorID != 'None'
GROUP BY DoctorID
Order By AvgDailyAppointments DESC""", conn)

,DoctorID,AvgDailyAppointments
0,D5,7.153846
1,D29,7.153846
2,D20,6.888889
3,D22,6.461538
4,D8,6.400000
5,D15,6.400000
6,D11,6.400000
7,D26,6.250000
8,D7,6.235294
9,D17,6.157895


##  Which departments have the 
## highest no-show rates?

In [30]:
pd.read_sql_query("""SELECT 
  Department,
  COUNT(*) FILTER (WHERE Status = 'No Show') * 100.0 / COUNT(*) AS NoShowRate
FROM Appointments
GROUP BY Department
ORDER BY NoShowRate DESC;""", conn)

,Department,NoShowRate
0,Orthopedics,40.760870
1,Oncology,29.770992
2,Neurology,26.470588
3,General Medicine,20.578778
4,Pediatric,15.676960
5,Cardiology,13.647643
6,None,0.000000


## Are certain doctors or departments 
## overbooked or underutilized?

In [31]:
pd.read_sql_query("""SELECT 
  DoctorID,
  Department,
  COUNT(*) / COUNT(DISTINCT Date) AS AvgAppointmentsPerDay
FROM Appointments
GROUP BY DoctorID, Department
ORDER BY AvgAppointmentsPerDay DESC;""", conn)

,DoctorID,Department,AvgAppointmentsPerDay
0,D20,General Medicine,14.0
1,D11,Neurology,10.0
2,D11,Orthopedics,10.0
3,D13,Orthopedics,10.0
4,D15,Pediatric,10.0
...,...,...,...
120,D9,Cardiology,5.0
121,D9,General Medicine,5.0
122,D9,Neurology,5.0
123,D9,Orthopedics,5.0


## What is the average number of 
## daily appointments per department?

In [32]:
pd.read_sql_query("""SELECT 
  Department,
  COUNT(*) * 1.0 / COUNT(DISTINCT Date) AS AvgDailyAppointments
FROM Appointments
Where Department != 'None'
GROUP BY Department;""", conn)

,Department,AvgDailyAppointments
0,Cardiology,6.296875
1,General Medicine,5.980769
2,Neurology,6.375000
3,Oncology,6.661017
4,Orthopedics,6.344828
5,Pediatric,5.847222


## What is the average feedback score 
## by department?

In [33]:
pd.read_sql_query("""SELECT 
    Department,
    AVG(Feedback_Score) AS AvgFeedbackScore
FROM feedback
GROUP BY Department
ORDER BY AvgFeedbackScore DESC;""", conn)

,Department,AvgFeedbackScore
0,Cardiology,7.030303
1,Oncology,6.500000
2,Pediatric,6.324324
3,Neurology,6.264706
4,Orthopedics,5.931034
5,General Medicine,5.384615


## Which departments receive the highest
## or lowest patient satisfaction?

In [34]:
pd.read_sql_query("""SELECT 
    Department,
    AVG(Feedback_Score) AS AvgFeedbackScore
FROM feedback
WHERE Department != 'None'
GROUP BY Department
ORDER BY AvgFeedbackScore DESC""", conn)

,Department,AvgFeedbackScore
0,Cardiology,7.030303
1,Oncology,6.500000
2,Pediatric,6.324324
3,Neurology,6.264706
4,Orthopedics,5.931034
5,General Medicine,5.384615


## Are there specific comments/themes 
## recurring in low feedback scores?

In [35]:
pd.read_sql_query("""SELECT 
    Comments
FROM feedback
WHERE Feedback_Score <= 3
ORDER BY FeedbackID LIMIT 10;""", conn)

,Comments
0,Needs improvement
1,Long wait times
2,Long wait times
3,Long wait times
4,Needs improvement
5,Needs improvement
6,Needs improvement
7,Long wait times
8,Needs improvement
9,Long wait times


## Which resources have the highest 
## and lowest usage hours by department?

In [36]:
pd.read_sql_query("""SELECT 
    Department,
    ResourceID,
    SUM(UsageHours) AS TotalUsageHours
FROM Resource
GROUP BY Department, ResourceID
ORDER BY Department, TotalUsageHours DESC;""", conn)

,Department,ResourceID,TotalUsageHours
0,Cardiology,R8641,9
1,Cardiology,R1998,9
2,Cardiology,R9923,8
3,Cardiology,R7116,8
4,Cardiology,R4245,8
...,...,...,...
189,Pediatric,R7483,0
190,Pediatric,R7354,0
191,Pediatric,R7325,0
192,Pediatric,R3242,0


## What is the average availability rate 
## of resources in each department?

In [37]:
pd.read_sql_query("""SELECT 
    Department,
    AVG(CASE WHEN Availability = 'Available' THEN 1 ELSE 0 END) * 100 AS AvgAvailabilityRate
FROM Resource
GROUP BY Department
ORDER BY AvgAvailabilityRate DESC;""", conn)

,Department,AvgAvailabilityRate
0,Cardiology,57.142857
1,Neurology,52.941176
2,Pediatric,50.000000
3,Orthopedics,42.307692
4,General Medicine,41.025641
5,Oncology,26.470588


## Which resource types are most prone 
## to unavailability or downtime?

In [38]:
pd.read_sql_query("""SELECT 
    ResourceType,
    COUNT(*) FILTER (WHERE Availability = 'Unavailable') * 100.0 / COUNT(*) AS DowntimeRate
FROM Resource
GROUP BY ResourceType
ORDER BY DowntimeRate DESC;""", conn)

,ResourceType,DowntimeRate
0,Doctor,54.761905
1,Nurse,51.428571
2,Technician,39.583333
3,Room,0.000000
4,Equipment,0.000000


## Is there a relationship between resource availability
## and appointment completion?

In [39]:
pd.read_sql_query("""SELECT 
    a.Department,
    AVG(CASE WHEN a.Status = 'Completed' THEN 1 ELSE 0 END) * 100 AS CompletionRate,
    AVG(CASE WHEN r.Availability = 'Available' THEN 1 ELSE 0 END) * 100 AS AvailabilityRate
FROM Appointments a
JOIN Resource r 
    ON a.Department = r.Department
    AND a.Date = r.Date
GROUP BY a.Department
ORDER BY AvailabilityRate DESC;""", conn)

,Department,CompletionRate,AvailabilityRate
0,Pediatric,27.272727,72.727273
1,Orthopedics,33.333333,50.000000
2,Oncology,50.000000,50.000000
3,Neurology,33.333333,41.666667
4,General Medicine,37.500000,37.500000
5,Cardiology,25.000000,25.000000


## Do departments with high appointment volume 
## have lower feedback scores?

In [41]:
pd.read_sql_query("""SELECT 
    a.Department,
    COUNT(a.Appointment) AS TotalAppointments,
    AVG(f.Feedback_Score) AS AvgFeedbackScore
FROM Appointments a
JOIN Feedback f 
    ON a.Department = f.Department
GROUP BY a.Department
ORDER BY TotalAppointments DESC;""", conn)

,Department,TotalAppointments,AvgFeedbackScore
0,Pediatric,14911,6.324324
1,Neurology,13566,6.264706
2,Cardiology,13299,7.030303
3,Oncology,11712,6.500000
4,Orthopedics,10672,5.931034
5,General Medicine,7384,5.384615


## Does resource usage efficiency correlate 
## with higher patient satisfaction?

In [42]:
pd.read_sql_query("""SELECT 
    r.Department,
    AVG(r.UsageHours) AS AvgUsageHours,
    AVG(f.Feedback_Score) AS AvgFeedbackScore
FROM Resource r
JOIN Feedback f 
    ON r.Department = f.Department
GROUP BY r.Department
ORDER BY AvgUsageHours DESC;""", conn)

,Department,AvgUsageHours,AvgFeedbackScore
0,Pediatric,4.800000,6.324324
1,Orthopedics,4.615385,5.931034
2,Cardiology,4.571429,7.030303
3,General Medicine,4.512821,5.384615
4,Neurology,4.294118,6.264706
5,Oncology,3.764706,6.500000


## Are patient wait times longer in departments 
## with lower resource availability?

In [43]:
pd.read_sql_query("""SELECT 
    f.Department,
    AVG(f.WaitTime) AS AvgWaitTime,
    AVG(CASE WHEN r.Availability = 'Available' THEN 1 ELSE 0 END) * 100 AS AvailabilityRate
FROM Feedback f
JOIN Resource r 
    ON f.Department = r.Department
GROUP BY f.Department
ORDER BY AvailabilityRate ASC;""", conn)

,Department,AvgWaitTime,AvailabilityRate
0,Oncology,39.375000,26.470588
1,General Medicine,43.846154,41.025641
2,Orthopedics,42.413793,42.307692
3,Pediatric,41.351351,50.000000
4,Neurology,42.794118,52.941176
5,Cardiology,40.000000,57.142857
